In [1]:
import scipy 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_input_folder = 'ISRwall/input/'
data_input_01 = scipy.io.loadmat(data_input_folder + 'kp_0002.mat')

data_input_01_desc = data_input_01['desc']
data_input_01_kp = data_input_01['kp']

counter = -1

data_01_desc = data_input_01_desc[:counter, :]
data_01_kp = data_input_01_kp[:counter, :]


In [3]:
print(data_01_desc.shape)
print(data_01_kp.shape)


print(data_input_01_desc.shape)
print(data_input_01_kp.shape)

(6807, 64)
(6807, 2)
(6808, 64)
(6808, 2)


In [4]:
data_ref_folder = 'ISRwall/reference/'
data_reference = scipy.io.loadmat(data_ref_folder + 'kp_ref.mat')


data_ref_desc = data_reference['desc']
data_ref_kp = data_reference['kp']

print(data_ref_desc.shape)
print(data_ref_kp.shape)


data_ref_desc = data_ref_desc[:counter, :]
data_ref_kp = data_ref_kp[:counter, :]

(6106, 64)
(6106, 2)


In [5]:
import cv2

In [32]:
class DMatch:
    def __init__(self, _queryIdx, _trainIdx, _distance):
        self.queryIdx = _queryIdx
        self.trainIdx = _trainIdx
        self.distance = _distance
    
    
    def __repr__(self):
        return 'DMatch(queryIdx: %s, trainIdx: %s, distance: %s)' % (self.queryIdx, self.trainIdx, self.distance)



# Function to calculate Euclidean distance
def euclidean_distance(descriptor1, descriptor2):
    return np.sqrt(np.sum((descriptor1 - descriptor2) ** 2))


# Function to find the K nearest neighbors
def knn_match(descriptors_query, descriptors_train, k):
    matches = []
    for i, query_descriptor in enumerate(descriptors_query):
        distances = []
        for j, train_descriptor in enumerate(descriptors_train):
            distance = euclidean_distance(query_descriptor, train_descriptor)
            distances.append((j, distance))
        
        distances.sort(key=lambda x: x[1])
        k_nearest_neighbors = distances[:k]
        match = [DMatch(_queryIdx=i, _trainIdx=neighbor[0], _distance=neighbor[1]) for neighbor in k_nearest_neighbors]
        matches.append(match)
    return matches



def knn_match_vectorized(descriptors_query, descriptors_train, k):
    distances = scipy.spatial.distance.cdist(descriptors_query, descriptors_train, 'euclidean') ## calculate the distances between each query descriptor and each train descriptor: i -> query, j -> train
    indices = np.argsort(distances, axis=1) ## order for each row (query descriptor) the indices of the train descriptors
    matches = np.zeros((descriptors_query.shape[0], k), dtype=object)
    
    for i in range(descriptors_query.shape[0]):
        k_nearest_neighbors = [(index, distances[i, index]) for index in indices[i, :k]]
        match = np.array([DMatch(_queryIdx=i, _trainIdx=neighbor[0], _distance=neighbor[1]) for neighbor in k_nearest_neighbors])
        matches[i, :] = match
        
    return matches


def ratio_test(matches, ratio=0.75):
    good_matches = []
    for m, n in matches:
        if m.distance < ratio * n.distance:
            good_matches.append(m)
            
    return np.array(good_matches)




In [7]:
bf = cv2.BFMatcher()
matches = bf.knnMatch(data_input_01_desc, data_ref_desc, k=2)  # Get the 2 nearest neighbors


good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)
        
        

In [8]:
input_quey = data_01_desc[:, :]
print(input_quey.shape)

(6807, 64)


In [33]:
bf = cv2.BFMatcher()
matches = bf.knnMatch(input_quey, data_ref_desc, k=2)  # Get the 2 nearest neighbors

good_matches = ratio_test(matches)

idx_query_vectorized = np.array([match.queryIdx for match in good_matches])
idx_train_vectorized = np.array([match.trainIdx for match in good_matches])

query_kp_vectorized = data_01_kp[idx_query_vectorized, :].astype(np.uint16)
train_kp_vectorized = data_ref_kp[idx_train_vectorized, :].astype(np.uint16)


# for idx in range(10):
    
#     first_match = good_matches[idx]
#     print("query_match: ", first_match.queryIdx)
#     print("train_match: ", first_match.trainIdx)
    
#     query_kp = data_01_kp[first_match.queryIdx]
#     train_kp = data_ref_kp[first_match.trainIdx]

#     print(f"Query KeyPoint: {query_kp}")
#     print(f"Train KeyPoint: {train_kp}")


In [34]:
print(query_kp_vectorized[:10])
print(train_kp_vectorized[:10])

[[1907 2014]
 [1970 1725]
 [2014 1961]
 [2083 1885]
 [2079 1860]
 [3196 1189]
 [2765 1838]
 [2130 1889]
 [2009 1862]
 [2334 1831]]
[[2295 2064]
 [1473  874]
 [1528 1108]
 [1596 1032]
 [1591 1008]
 [2722  298]
 [2295 2064]
 [1644 1035]
 [1520 1011]
 [1849  973]]


In [10]:
# matches2 = knn_match(input_quey, data_ref_desc, 2)

# good_matches2 = []
# for m, n in matches2:
#     if m.distance < 0.75 * n.distance:
#         good_matches2.append(m)


# for idx in range(len(good_matches2)):
    
#     first_match = good_matches2[idx]
#     print("query_match: ", first_match.queryIdx)
#     print("train_match: ", first_match.trainIdx)
    
#     query_kp = data_01_kp[first_match.queryIdx]
#     train_kp = data_ref_kp[first_match.trainIdx]

#     print(f"Query KeyPoint: {query_kp}")
#     print(f"Train KeyPoint: {train_kp}")

In [35]:
matches3 = knn_match_vectorized(input_quey, data_ref_desc, 2)

good_matches = ratio_test(matches3)

idx_query_vectorized = np.array([match.queryIdx for match in good_matches])
idx_train_vectorized = np.array([match.trainIdx for match in good_matches])

query_kp_vectorized = data_01_kp[idx_query_vectorized, :].astype(np.uint16)
train_kp_vectorized = data_ref_kp[idx_train_vectorized, :].astype(np.uint16)



# for idx in range(len(good_matches3)):
    
#     first_match = good_matches3[idx]
#     print("query_match: ", first_match.queryIdx)
#     print("train_match: ", first_match.trainIdx)
    
#     query_kp = data_01_kp[first_match.queryIdx]
#     train_kp = data_ref_kp[first_match.trainIdx]

#     print(f"Query KeyPoint: {query_kp}")
#     print(f"Train KeyPoint: {train_kp}")

In [12]:
print("Matches: ", matches3.shape)

Matches:  (6807, 2)
